<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/text-similarity-works/icd10-code-highlighting/13_3_icd_10_9_code_and_keyword_impairment_highlighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [1]:
%%capture

!pip -q install spacy
!python -m spacy download en_core_web_sm

!pip install pillow

!pip install more-itertools
!pip install PyPDF2
!pip install pymupdf==1.22.0
!pip install fuzzywuzzy

In [ ]:
import fitz
print(fitz.__doc__)


PyMuPDF 1.22.0: Python bindings for the MuPDF 1.22.0 library.
Version date: 2023-04-14 00:00:01.
Built for Python 3.10 on linux (64-bit).



In [ ]:
import more_itertools
print(more_itertools.__version__)

10.1.0


In [ ]:
#!pip install pikepdf
#!apt install ocrmypdf

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!rm -rf data
!unzip data.zip

##Import

In [1]:
import re
import os
import shutil
import glob
import json
import logging
import more_itertools
from pathlib import Path

import numpy as np
import pandas as pd

import fitz

from spacy.lang.en import English

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz

from dataclasses import dataclass
from array import *

from concurrent import futures
from highlighting import Highlighter
from keyword_matcher import KeywordMatcher
from sentence_extractor import SentenceExtractor
from generic_matcher import GenericMatcher
import config as cfg
from utils import *

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


##Core Classes

In [ ]:
(238/255, 210/255, 2/255)

(0.9333333333333333, 0.8235294117647058, 0.00784313725490196)

In [ ]:
class Highlighter:
    def __init__(self, match_threshold=30):
        # loading and updating patterns for ICD-10 code
        self.nlp_code10 = English()
        self.nlp_code10.add_pipe("entity_ruler").from_disk(cfg.pattern_files["icd_10_jsonl"])

        # loading and updating patterns for ICD-9 code
        if cfg.load_pattern["icd_code9"]:
            self.nlp_code9 = English()
            self.nlp_code9.add_pipe("entity_ruler").from_disk(cfg.pattern_files["icd_9_jsonl"])

        self.match_threshold = match_threshold
        self.impairment_keyword_dict = None
        self.page_matched_date_dict = None
        self.page_matched_number_dict = None
        self.text_list = None

        # define icd-10 code dataset
        core_df = pd.read_csv(cfg.csv_files["CODE_CSV"])
        self.code_df = core_df.where(pd.notnull(core_df), None)
        self.synid_df = pd.read_csv(cfg.csv_files["IMP_CSV"])

        # define required directory path
        self.PDF_FILES_PATH = cfg.file_path["TMP_PDF_FILES_PATH"]
        self.TXT_FILES_PATH = cfg.file_path["TMP_TXT_FILES_PATH"]
        self.OUTPUT_FILES_PATH = cfg.file_path["OUTPUT_PATH"]

        self.logger = logging.getLogger(__name__)

    def set_impairment_keyword_dict(self, imp_keyword_dict):
        self.impairment_keyword_dict = imp_keyword_dict

    def set_page_matched_date_dict(self, matched_date_dict):
        self.page_matched_date_dict = matched_date_dict

    def set_page_matched_number_dict(self, matched_number_dict):
        self.page_matched_number_dict = matched_number_dict

    def set_match_threshold(self, match_val):
        self.match_threshold = match_val

    def set_text_list(self, p_text_list):
        self.text_list = p_text_list

    def highlight_all(self, icd10_code_dict, pdf_file_name=None, highlight_all=False):
        pdf_file = fitz.open(pdf_file_name)
        json_data = []
        json_imp_data = []
        json_date_data = []

        for page_num, page in enumerate(pdf_file):
            already_done_page_list = []
            self.code_y2_coords_list = []
            self.already_found_word_list = []
            self.highlighted_coord_list = []
            code_highlight_dict = {}

            # highlight ICD-10 code
            if page_num in icd10_code_dict:
                num_page = page_num + 1

                # let's store code y2 coords
                for code in icd10_code_dict[page_num]:
                    highlight_list = page.search_for(code)
                    #print(f"Page-{num_page}-{code}, Coordinate: {highlight_list}")
                    if len(highlight_list) > 0:
                        code_highlight_dict[code] = highlight_list
                        self.code_y2_coords_list.extend([highlight[3] for highlight in highlight_list])
                    else:
                        for alt_code in get_alternate_code_pattern(code):
                            highlight_list = page.search_for(alt_code)
                            for highlight in highlight_list:
                                if len(highlight) > 0:
                                    code_highlight_dict[code] = [highlight]
                                    # print(f"alternate-code:{code}, Coordinate: {highlight}")
                                    self.code_y2_coords_list.extend([highlight[3]])
                                    # print(f"[highlight[3]]: {[highlight[3]]}")

                filtered_code_highlight_dict = filter_duplicate_coordinate(code_highlight_dict)
                # now, let highlight every code and its common words
                for code, highlight_list in filtered_code_highlight_dict.items():
                    prev_highlight = fitz.Rect(0, 0, 0, 0)
                    for highlight in highlight_list:
                        keyword = get_keyword(code, self.code_df)
                        # get match score and common words coordinate
                        if not highlight_all:
                          all_match_keyword_list, all_match_imp_keyword_list = self.get_best_token_match(page_num,
                                                                                                        code,
                                                                                                        page,
                                                                                                        highlight[3],
                                                                                                        self.match_threshold)
                        else:
                          all_match_keyword_list, all_match_imp_keyword_list = self.get_best_all_token_match(page_num,
                                                                                                        code,
                                                                                                        page,
                                                                                                        highlight[3],
                                                                                                        self.match_threshold)

                        # Step-1: highlight and set color coding dont have common words
                        if not all_match_keyword_list:
                            highlight_code(page, highlight)
                            # prepare json object
                            json_data.append(prepare_json_object(num_page, code, self.code_df, highlight, keyword))

                        # Step-2: highlight and set color coding that have common words
                        for match_keyword_dict in all_match_keyword_list:
                            # highlight ICD-10 code
                            if not is_coord_equal(prev_highlight, highlight):
                              highlight_code(page, highlight)
                              prev_highlight = highlight
                            # highlight common words
                            highlight_coords_dict = {}
                            highlight_coords_dict, highlighted_word_list = self.highlight_common_words(page, match_keyword_dict["common_words_coords"])
                            # prepare json object
                            json_data.append(prepare_json_object(num_page, code, self.code_df, highlight, keyword, match_keyword_dict, highlight_coords_dict))

                        # Step-3: highlight and set color coding for impairment keyword common words
                        json_imp_data_list = []
                        for match_imp_keyword_dict in all_match_imp_keyword_list:
                            json_imp_data_object = {}
                            # highlight common words
                            highlight_coords_dict = {}
                            if "imp_common_keyword_coords" in match_imp_keyword_dict:
                              if match_imp_keyword_dict["imp_common_keyword_coords"]:
                                highlight_coords_list = self.highlight_impairment(page, num_page, match_imp_keyword_dict["imp_common_keyword_coords"])
                                already_done_page_list.append(page_num)
                                # prepare json object
                                if highlight_coords_list:
                                  json_imp_data_list.extend(highlight_coords_list)
                        if len(json_imp_data_list) > 0:
                            json_imp_data.extend(json_imp_data_list)

            # highlight impairment key phrase
            if page_num in self.impairment_keyword_dict and page_num not in already_done_page_list:
              json_imp_data_list = self.highlight_keyword_impairment_data(page, page_num)
              if len(json_imp_data_list) > 0:
                json_imp_data.extend(json_imp_data_list)

            # highlight date and time in PDF
            if page_num in self.page_matched_date_dict:
                json_date_data_list = self.highlight_date_and_time(page, page_num)
                if len(json_date_data_list) > 0:
                  json_date_data.extend(json_date_data_list)

        # save PDF and JSON output file
        pdf_output_file_name, json_data_dump_file = save_output_file(pdf_object=pdf_file, cfg=cfg,
                                                                     output_type=2 if highlight_all else 1,
                                                                     output_path=self.OUTPUT_FILES_PATH,
                                                                     output_file_name=pdf_file_name,
                                                                     json_data=json_data,
                                                                     json_imp_data=json_imp_data,
                                                                     json_date_data=json_date_data)
        return pdf_output_file_name, json_data_dump_file

    def highlight_icd_code_and_common_words(self, icd10_code_dict, pdf_file_name=None):
        pdf_file = fitz.open(pdf_file_name)
        json_data = []

        for page_num, page in enumerate(pdf_file):
            self.code_y2_coords_list = []
            self.already_found_word_list = []
            code_highlight_dict = {}

            # highlight ICD-10 code
            if page_num in icd10_code_dict:
                num_page = page_num + 1
                # let's store code y2 coords
                for code in icd10_code_dict[page_num]:
                    highlight_list = page.search_for(code)
                    #print(f"Page-{num_page}-{code}, Coordinate: {highlight_list}")
                    if len(highlight_list) > 0:
                        code_highlight_dict[code] = highlight_list
                        self.code_y2_coords_list.extend([highlight[3] for highlight in highlight_list])
                    else:
                        for alt_code in get_alternate_code_pattern(code):
                            highlight_list = page.search_for(alt_code)
                            for highlight in highlight_list:
                                if len(highlight) > 0:
                                    code_highlight_dict[code] = [highlight]
                                    #print(f"alternate-code:{code}, Coordinate: {highlight}")
                                    self.code_y2_coords_list.extend([highlight[3]])

                filtered_code_highlight_dict = filter_duplicate_coordinate(code_highlight_dict)
                # now, let highlight every code and its common words
                for code, highlight_list in filtered_code_highlight_dict.items():
                    prev_highlight = fitz.Rect(0, 0, 0, 0)
                    for highlight in highlight_list:
                        keyword = get_keyword(code, self.code_df)
                        # get match score and common words coordinate
                        all_match_keyword_list, all_match_imp_keyword_list = self.get_best_token_match(page_num, code,
                                                                                                       page,
                                                                                                       highlight[3],
                                                                                                       self.match_threshold)

                        # Step-1: highlight and set color coding dont have common words
                        if not all_match_keyword_list:
                            page_highlight = page.add_highlight_annot(highlight)
                            page_highlight.set_colors(stroke=[0.92, 0.59, 0.48])  # dark salmon
                            page_highlight.update()
                            # prepare json object
                            json_data.append(prepare_json_object(num_page, code, self.code_df, highlight, keyword))

                        # Step-2: highlight and set color coding that have common words
                        for match_keyword_dict in all_match_keyword_list:
                            # highlight ICD-10 code
                            if not self.is_coord_equal(prev_highlight, highlight):
                              highlight_code(page, highlight)
                              prev_highlight = highlight
                            # highlight common words
                            highlight_coords_dict = {}
                            highlight_coords_dict, highlighted_word_list = self.highlight_common_words(page, match_keyword_dict["common_words_coords"])

                            # prepare json object
                            json_data.append(prepare_json_object(num_page, code, self.code_df, highlight, keyword, match_keyword_dict, highlight_coords_dict))

        # save PDF and JSON output file
        pdf_output_file_name, json_data_dump_file = save_output_file(pdf_file, cfg, 3, self.OUTPUT_FILES_PATH, pdf_file_name, json_data)
        return pdf_output_file_name, json_data_dump_file

    def highlight_icd_code_and_keyword_impairment(self, icd10_code_dict, icd9_code_dict=None, pdf_file_name=None):
      pdf_file = fitz.open(pdf_file_name)
      json_data = []
      json_imp_data = []
      json_date_data = []
      json_number_data = []

      for page_num, page in enumerate(pdf_file):
        already_done_page_list = []
        self.code_y2_coords_list = []
        self.already_found_word_list = []
        self.highlighted_coord_list = []
        self.highlighted_term_list = []
        code_highlight_dict = {}

        # highlight ICD-10 code
        if page_num in icd10_code_dict:
            num_page = page_num + 1

            # let's store code y2 coords
            for code in icd10_code_dict[page_num]:
                highlight_list = page.search_for(code)
                # print(f"Page-{num_page}-{code}, Coordinate: {highlight_list}")
                if len(highlight_list) > 0:
                    code_highlight_dict[code] = highlight_list
                    self.code_y2_coords_list.extend([highlight[3] for highlight in highlight_list])
                else:
                    for alt_code in get_alternate_code_pattern(code):
                        highlight_list = page.search_for(alt_code)
                        for highlight in highlight_list:
                            if len(highlight) > 0:
                                code_highlight_dict[code] = [highlight]
                                # print(f"alternate-code:{code}, Coordinate: {highlight}")
                                self.code_y2_coords_list.extend([highlight[3]])
                                # print(f"[highlight[3]]: {[highlight[3]]}")
            # print(f"code_highlight_dict: {code_highlight_dict}")
            filtered_code_highlight_dict = filter_duplicate_coordinate(code_highlight_dict)
            # now, let highlight every code and its common words
            for code, highlight_list in filtered_code_highlight_dict.items():
                # print(f"code-{code}, Coordinate: {highlight_list}")
                prev_highlight = fitz.Rect(0, 0, 0, 0)
                for highlight in highlight_list:
                    # Step-1: highlight ICD-10 code
                    #print(f"prev_highlight: {prev_highlight}")
                    if not is_coord_equal(prev_highlight, highlight):
                      #print(f"highlight: {highlight}")
                      highlight_code(page, highlight)
                      prev_highlight = highlight
                    # prepare json object
                    json_data.append(prepare_code_json_object(num_page, code, highlight))

        # highlight ICD-9 code
        if icd9_code_dict is not None:
            if page_num in icd9_code_dict:
              num_page = page_num + 1
              for code in icd9_code_dict[page_num]:
                highlight_list = page.search_for(code)
                """
                if len(highlight_list) == 0:
                    alternate_code_list = get_optional_pattern(code)
                    for alt_code in alternate_code_list:
                        highlights = page.search_for(alt_code)
                        # highlight pdf for optional pattern
                        for highlight in highlights:
                          if is_exact_match(page, alt_code, highlight, full_match=True, case_sensitive=True):
                            highlight_code(page, highlight, code_type="ICD-9")
                            json_data.append(prepare_code_json_object(num_page, alt_code, highlight, code_type="ICD-9"))
                """
                # highlight pdf for main pattern
                for highlight in highlight_list:
                    if is_exact_code_match(page, highlight):
                      highlight_code(page, highlight, code_type="ICD-9")
                      # prepare json object
                      json_data.append(prepare_code_json_object(num_page, code, highlight, code_type="ICD-9"))

        # highlight keyword impairment phrase
        # if page_num in self.impairment_keyword_dict and page_num not in already_done_page_list:
        if page_num in self.impairment_keyword_dict:
          json_imp_data_list = self.highlight_keyword_impairment_data(page, page_num)
          if len(json_imp_data_list) > 0:
            json_imp_data.extend(json_imp_data_list)

        # highlight date and time in PDF
        if page_num in self.page_matched_date_dict:
          json_date_data_list = self.highlight_date_and_time(page, page_num)
          if len(json_date_data_list) > 0:
            json_date_data.extend(json_date_data_list)

        # highlight number data in PDF
        if page_num in self.page_matched_number_dict:
            json_number_data_list = self.highlight_number_data(page, page_num)
            if len(json_number_data_list) > 0:
                json_number_data.extend(json_number_data_list)

      # save PDF and JSON output file
      pdf_output_file_name, json_data_dump_file = save_output_file(pdf_object=pdf_file, cfg=cfg,
                                                                   output_type=4,
                                                                   output_path=self.OUTPUT_FILES_PATH,
                                                                   output_file_name=pdf_file_name,
                                                                   json_data=json_data,
                                                                   json_imp_data=json_imp_data,
                                                                   json_date_data=json_date_data,
                                                                   json_number_data=json_number_data)
      return pdf_output_file_name, json_data_dump_file

    def highlight_keyword_impairment(self, pdf_file_name=None):
        pdf_file = fitz.open(pdf_file_name)
        json_imp_data = []
        for page_num, page in enumerate(pdf_file):
          self.highlighted_coord_list = []
          # highlight ICD key phrase
          if page_num in self.impairment_keyword_dict:
            json_imp_data_list = []
            num_page = page_num + 1
            imp_keywords_coord_dict = {}
            key_phrase_sents = self.impairment_keyword_dict[page_num]
            for key_phrase_sent in key_phrase_sents:
              #print(f"key_phrase_sent: {key_phrase_sent}")
              coordinates = page.search_for(key_phrase_sent)
              imp_keywords_coord_dict[key_phrase_sent] = coordinates
            highlight_coords_list = self.highlight_impairment(page, num_page, imp_keywords_coord_dict)
            if len(highlight_coords_list) > 0:
              json_imp_data_list.extend(highlight_coords_list)
            if len(json_imp_data_list) > 0:
              json_imp_data.extend(json_imp_data_list)

        # save PDF and JSON output file
        pdf_output_file_name, json_data_dump_file = save_output_file(pdf_file, cfg, 5, self.OUTPUT_FILES_PATH, pdf_file_name, json_imp_data=json_imp_data)
        return pdf_output_file_name, json_data_dump_file

    def highlight_keyword_impairment_data(self, p_page, page_num):
      json_imp_data_list = []
      num_page = page_num + 1
      # print(f"num_page: {num_page}")
      imp_keywords_coord_dict = {}

      def highlight_multi_key_phrase(p_keyword_impairment, batch_coord):
        phrase_coords = []
        filter_coords = []
        try:
            f_coord = batch_coord[0]
            l_coord = batch_coord[-1]
            # first coord y1,y2 and second coord y1,y2 must be equal if it is same phrase
            if f_coord[1] == l_coord[1] and f_coord[3] == l_coord[3]:
              # check first coord x2 and second coord x1 is more than 100(means the same line phrase occurrence)
              if (l_coord[0] - f_coord[2]) > 100:
                  phrase_coords.extend([
                      (f_coord[0], f_coord[1], f_coord[2], f_coord[3]),
                      (l_coord[0], l_coord[1], l_coord[2], l_coord[3])
                  ])
              else:
                phrase_coords.extend([(f_coord[0], f_coord[1], l_coord[2], l_coord[3])])
            for phrase_coord in phrase_coords:
                # print(f"phrase_coord: {phrase_coord}")
                if is_exact_match(p_page, p_keyword_impairment, phrase_coord, full_match=True, case_sensitive=True) \
                  and not is_coord_already_highlighted(self.highlighted_coord_list, phrase_coord):
                  p_highlight = p_page.add_highlight_annot(phrase_coord)
                  p_highlight.update()
                  filter_coords.append(phrase_coord)
                  self.highlighted_coord_list.append(phrase_coord)
                # print(f"key_phrase: {p_keyword_impairment}, phrase_coord: {phrase_coord}")
        except ValueError as ve:
            #print(f"ERROR: coordinate not found>>{ve}")
            self.logger.error(f"ERROR: coordinate not found>>{ve}")
        return filter_coords

      def get_json_data(coords, k_impairment):
        return {
            "coords": coords,
            "synid": get_synid(k_impairment, self.synid_df),
            # "keyword_field": get_keyword_field(key_phrase_sent, synid_df)
          }

      keyword_impairment_dict = self.impairment_keyword_dict[page_num]
      for keyword_impairment in keyword_impairment_dict.keys():
          # print(f"key_phrase_sent: {keyword_impairment}")
          coordinates = p_page.search_for(keyword_impairment)
          # print(f"coordinates: {coordinates}")

          phrase_len = len(keyword_impairment.split())
          if coordinates:
            # highlight word phrase that have equal word phrase and coord length(single phrase occurrence)
            if phrase_len == len(coordinates) and phrase_len != 1:
                imp_keywords_coord_dict[keyword_impairment] = get_json_data(highlight_multi_key_phrase(keyword_impairment, coordinates), keyword_impairment_dict[keyword_impairment])
            # highlight word phrase that do not have equal word phrase and coord length(multiple phrase occurrence)
            elif int(len(coordinates) / phrase_len) * phrase_len == len(coordinates):
                phrase_coord_list = []
                # batch coord list by dividing its length with phrase length
                for batch_coord in more_itertools.batched(coordinates, phrase_len):
                    phrase_coord_list.extend(highlight_multi_key_phrase(keyword_impairment, batch_coord))
                imp_keywords_coord_dict[keyword_impairment] = get_json_data(phrase_coord_list, keyword_impairment_dict[keyword_impairment])
            else:
                coords_list = []
                for coord in coordinates:
                    if is_exact_match(p_page, keyword_impairment, coord, full_match=True, case_sensitive=True) and not is_coord_already_highlighted(self.highlighted_coord_list, coord):
                        page_highlight = p_page.add_highlight_annot(coord)
                        page_highlight.update()
                        coords_list.extend((coord[0], coord[1], coord[2], coord[3]))
                        self.highlighted_coord_list.append(coord)
                        # print(f"key_phrase: {keyword_impairment}, keywords_coord: {coord}")
                # print(f"coords_list: {coords_list}")
                imp_keywords_coord_dict[keyword_impairment] = get_json_data(coords_list, keyword_impairment_dict[keyword_impairment])
      # print(f"imp_keywords_coord_dict: {imp_keywords_coord_dict}")
      if imp_keywords_coord_dict:
        json_imp_data_list.extend(prepare_imp_json_object(num_page, imp_keywords_coord_dict))
      return json_imp_data_list

    def highlight_date_and_time(self, p_page, page_num):
        num_page = page_num + 1
        json_date_data_list = []
        highlight_coords_list = []
        page_date_coord_dict = {}

        def highlight_date(p_coordinate, p_date):
          page_highlight = p_page.add_highlight_annot(p_coordinate)
          page_highlight.set_colors(stroke=[0.529, 0.807, 0.921])  # sky blue
          page_highlight.update()
          page_date_coord_dict[p_date] = [p_coordinate[0], p_coordinate[1], p_coordinate[2], p_coordinate[3]]
          self.highlighted_term_list.append(p_date)

        page_date_list = self.page_matched_date_dict[page_num]
        for page_date in page_date_list:
            coordinates = p_page.search_for(page_date)
            for coordinate in coordinates:
              if len(re.findall(r"^\d+$", page_date.strip())) > 0:
                if is_exact_year_match(p_page, page_date, coordinate, self.highlighted_term_list):
                  highlight_date(coordinate, page_date)
              else:
                highlight_date(coordinate, page_date)

        if page_date_coord_dict:
            highlight_coords_list.append(prepare_generic_json_object(num_page, page_date_coord_dict))
        if len(highlight_coords_list) > 0:
            json_date_data_list.extend(highlight_coords_list)
        return json_date_data_list

    def highlight_number_data(self, p_page, page_num):
        num_page = page_num + 1
        json_number_data_list = []
        highlight_coords_list = []
        page_number_coord_dict = {}
        # real number regex
        real_number_regex = "^(?:-(?:[1-9](?:\\d{0,2}(?:,\\d{3})+|\\d*))|(?:0|(?:[1-9](?:\\d{0,2}(?:,\\d{3})+|\\d*))))(?:.\\d+|)$"

        page_number_list = self.page_matched_number_dict[page_num]
        for page_number in page_number_list:
            coordinates = p_page.search_for(page_number)
            for coordinate in coordinates:
                if len(re.findall(rf"{real_number_regex}", page_number.strip())) > 0:
                    if is_exact_number_match(p_page, page_number, coordinate):
                        page_highlight = p_page.add_highlight_annot(coordinate)
                        page_highlight.set_colors(stroke=[0.529, 0.907, 0.421])  # sky blue
                        page_highlight.update()
                        page_number_coord_dict[page_number] = [coordinate[0], coordinate[1], coordinate[2], coordinate[3]]

        if page_number_coord_dict:
            highlight_coords_list.append(prepare_generic_json_object(num_page, page_number_coord_dict))
        if len(highlight_coords_list) > 0:
            json_number_data_list.extend(highlight_coords_list)
        return json_number_data_list

    def highlight_common_words(self, page_obj, common_words_coord_dict):
        highlight_coords_dict = {}
        highlighted_word_list = []

        prev_common_word = ""
        for common_word, common_word_coord in common_words_coord_dict.items():
            highlight = page_obj.add_highlight_annot(common_word_coord)
            highlight.update()
            if prev_common_word.lower() != common_word.lower():
                highlight_coords_dict[common_word] = common_word_coord
                highlighted_word_list.append(common_word)
                prev_common_word = common_word
        return highlight_coords_dict, highlighted_word_list

    def get_best_token_match(self, page_num, p_code, page_obj, code_y2_coord, match_threshold):
        all_match_keyword_list = []
        all_match_imp_keyword_list = []

        # Step 1: reverse code pattern
        reversed_icd_code = reverse_code_pattern(p_code)
        # Step 2: fetch keyword based on code
        keyword = get_keyword(reversed_icd_code, self.code_df)
        # Step 3: get code paragraph
        code_paragraph_list, non_code_paragraph_list = get_paragraph(page_obj, code_y2_coord)
        # print(f"code_paragraph_list: {non_code_paragraph_list}")

        # Step 4: prepare code_paragraph for common words
        for code_paragraph in code_paragraph_list:
            match_keyword_dict = {}
            common_words = get_common_words(keyword, code_paragraph)
            if len(common_words) > 0:
                match_keyword_dict["common_words"] = common_words
                # Step 4: get best token match ratio
                clean_paragraph = " ".join(clean_text(code_paragraph))
                match_keyword_dict["paragraph"] = clean_paragraph
                # match_score = fuzz.token_set_ratio(keyword, clean_paragraph)
                # if fuzz.token_set_ratio(keyword, clean_paragraph) >= match_threshold else 0
                match_keyword_dict["score"] = fuzz.token_set_ratio(keyword, clean_paragraph)
                # Step 5: build common words coordinate dict
                common_words_coord_dict = {}
                #print(f"code_y2_coord: {code_y2_coord}")
                for common_word in common_words:
                    highlight_list = page_obj.search_for(common_word)
                    #print(f"highlight_list: {highlight_list}")
                    found_coord = False
                    for highlight in highlight_list:
                        # get common word y2 coord value
                        common_word_y2_coords = highlight[3]
                        common_word_y1_coords = highlight[1]
                        if (code_y2_coord - 2) <= common_word_y2_coords <= (code_y2_coord + 2):
                          common_words_coord_dict[common_word] = highlight
                          found_coord = True
                        if not found_coord:
                          if (code_y2_coord - 20) <= common_word_y2_coords <= (code_y2_coord + 20):
                            common_words_coord_dict[common_word] = highlight
                            # self.already_found_word_list.append(common_word)
                match_keyword_dict["common_words_coords"] = common_words_coord_dict
                all_match_keyword_list.append(match_keyword_dict)

        def get_keyword_impairment_list(p_paragraph):
            keyword_impairments = []
            for keyword_impairment in self.impairment_keyword_dict[page_num]:
                keyword_impairment_found = get_common_words(keyword_impairment, p_paragraph)
                if len(keyword_impairment_found) > 0:
                    keyword_impairments.append(keyword_impairment)
            return keyword_impairments

        # Step 5: prepare non code_paragraph for common words
        for non_code_paragraph in non_code_paragraph_list:
            match_imp_keyword_dict = {}
            imp_common_words_coord_dict = {}
            keyword_impairment_list = get_keyword_impairment_list(non_code_paragraph)
            for keyword_impairment in keyword_impairment_list:
                highlight_list = page_obj.search_for(keyword_impairment)
                highlight_list = [highlight for highlight in highlight_list if highlight[3] not in self.code_y2_coords_list]
                if keyword_impairment not in self.already_found_word_list and len(highlight_list) > 0:
                    self.already_found_word_list.append(keyword_impairment)
                    imp_common_words_coord_dict[f"{keyword_impairment}"] = highlight_list
                    continue
            match_imp_keyword_dict["imp_common_keyword_coords"] = imp_common_words_coord_dict
            all_match_imp_keyword_list.append(match_imp_keyword_dict)

        return all_match_keyword_list, all_match_imp_keyword_list

    def get_best_all_token_match(self, page_num, p_code, page_obj, code_y2_coord, match_threshold):
        all_match_keyword_list = []
        all_match_imp_keyword_list = []

        # Step 1: reverse code pattern
        reversed_icd_code = reverse_code_pattern(p_code)
        # Step 2: fetch keyword based on code
        keyword = get_keyword(reversed_icd_code, self.code_df)
        # Step 3: get all paragraph
        all_paragraph_list = get_all_paragraph(page_obj)
        # print(f"code_paragraph_list: {non_code_paragraph_list}")

        def get_keyword_impairment_list(p_paragraph):
          keyword_impairments = []
          for keyword_impairment in self.impairment_keyword_dict[page_num]:
              keyword_impairment_found = get_common_words(keyword_impairment, p_paragraph)
              if len(keyword_impairment_found) > 0:
                  keyword_impairments.append(keyword_impairment)
          return keyword_impairments

        # Step 4: prepare all_paragraph for common words and impairment
        for all_paragraph in all_paragraph_list:
          match_keyword_dict = {}
          common_words = get_common_words(keyword, all_paragraph)
          if len(common_words) > 0:
              match_keyword_dict["common_words"] = common_words
              # Step 4: get best token match ratio
              clean_paragraph = " ".join(clean_text(all_paragraph))
              match_keyword_dict["paragraph"] = clean_paragraph
              # match_score = fuzz.token_set_ratio(keyword, clean_paragraph)
              # if fuzz.token_set_ratio(keyword, clean_paragraph) >= match_threshold else 0
              match_keyword_dict["score"] = fuzz.token_set_ratio(keyword, clean_paragraph)
              # Step 5: build common words coordinate dict
              common_words_coord_dict = {}
              for common_word in common_words:
                  highlight_list = page_obj.search_for(common_word)
                  found_coord = False
                  for highlight in highlight_list:
                      # get common word y2 coord value
                      common_word_y2_coords = highlight[3]
                      if (code_y2_coord - 2) <= common_word_y2_coords <= (code_y2_coord + 2):
                        common_words_coord_dict[common_word] = highlight
                        found_coord = True
                      if not found_coord:
                        if (code_y2_coord - 20) <= common_word_y2_coords <= (code_y2_coord + 20):
                          common_words_coord_dict[common_word] = highlight
                          # self.already_found_word_list.append(common_word)
              match_keyword_dict["common_words_coords"] = common_words_coord_dict
              all_match_keyword_list.append(match_keyword_dict)
          else:
            # Step 5: prepare non code_paragraph for common words
            match_imp_keyword_dict = {}
            imp_common_words_coord_dict = {}
            keyword_impairment_list = get_keyword_impairment_list(all_paragraph)
            for keyword_impairment in keyword_impairment_list:
              #print(f"keyword_impairment11: {keyword_impairment}")
              highlight_list = page_obj.search_for(keyword_impairment)
              if keyword_impairment not in self.already_found_word_list and len(highlight_list) > 0:
                self.already_found_word_list.append(keyword_impairment)
                imp_common_words_coord_dict[f"{keyword_impairment}"] = highlight_list
                #print(f"keyword_impairment22: {keyword_impairment}, coords: {highlight_list}")
                #continue
            match_imp_keyword_dict["imp_common_keyword_coords"] = imp_common_words_coord_dict
            all_match_imp_keyword_list.append(match_imp_keyword_dict)

        return all_match_keyword_list, all_match_imp_keyword_list

    def search_icd_code(self, txt_list, code_type="ICD-10"):
      pdf_page_vocab = {}
      for txt_file in txt_list:
        with open(txt_file, "r") as f:
          page_txt = f.read()
          # check the page that have line number instead of code
          index_page = False
          if re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
            index_page = True

          if code_type == "ICD-9":
            doc = self.nlp_code9(page_txt)
          else:
            doc = self.nlp_code10(page_txt)

          code_list = []
          if index_page:
            # check the code contain letter "L"
            code_list = [ent.text for ent in doc.ents if not re.search("(L[0-9]+)", ent.text)]
          else:
            code_list = [ent.text for ent in doc.ents]

          if len(code_list) != 0:
            # page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
            page_number = int(txt_file.split("/")[-1].split(".")[0].split("-")[1])
            pdf_page_vocab[page_number] = list(set(code_list))
            # print(f"Page[{txt_file.split('/')[1]}]: {code_list}")
      return pdf_page_vocab

##PDF Highlighting

In [2]:
!mkdir -p /home/ocreng
!mkdir -p /home/ocreng/ocrhigh
!mkdir -p /home/ocreng/ocrhigh/input
!mkdir -p /home/ocreng/ocrhigh/output
!mkdir -p /home/ocreng/ocrhigh/processed
!mkdir -p /home/ocreng/ocrhigh/pdf-files
!mkdir -p /home/ocreng/ocrhigh/txt-files

In [3]:
def purge(file_path):
  for f in glob.glob(file_path):
    os.remove(f)

In [4]:
%%time
# Step-0: create highlighter instance
INPUT_PDF_FILES_PATH = cfg.file_path["INPUT_PATH"]

highlighter = Highlighter(match_threshold=35)
sent_extractor = SentenceExtractor()
generic_matcher = GenericMatcher()

CPU times: user 1min 3s, sys: 4.25 s, total: 1min 8s
Wall time: 1min 11s


```log
CPU times: user 1min 7s, sys: 6.54 s, total: 1min 14s
Wall time: 1min 15s
```

##Test

In [13]:
!rm -rf /home/ocreng/ocrhigh/input
!mkdir -p /home/ocreng/ocrhigh/input

In [14]:
!cp Practice_Copy_Test_OCR.pdf /home/ocreng/ocrhigh/input
#!cp quinn_ian_michael_5353_1368260963_awp_30a71784-41b8-423d-9d29-8f338f050398.pdf /home/ocreng/ocrhigh/input

In [15]:
!rm -rf /home/ocreng/ocrhigh/output
!mkdir -p /home/ocreng/ocrhigh/output
!rm -rf output*.zip

In [16]:
%%time
text_list = None
icd10_code_dict1 = None
icd9_code_dict1 = None
wrong_keyword_dict = None
for pdf_file in os.listdir(INPUT_PDF_FILES_PATH):
    print(f"#####################Processing File[{pdf_file}] for highlighting###########################")
    pdf_file_name = f"{INPUT_PDF_FILES_PATH}/{pdf_file}"

    # Step-1: splitting pdf file
    print("Step-1: Splitting pdf file............")
    pdf_list = split_pdf(highlighter.PDF_FILES_PATH, pdf_file_name)

    # Step-2: Extracting text from pdf
    print("Step-2: Extracting text from pdf............")
    txt_list = extract_text_from_pdf(highlighter.PDF_FILES_PATH, highlighter.TXT_FILES_PATH, pdf_list)
    highlighter.set_text_list(txt_list)

    # Step-3: Searching ICD-10 cod
    print("Step-3: Searching ICD-10 code into text file..........")
    icd10_code_dict = highlighter.search_icd_code(txt_list)

    # Step-4: Searching ICD-9 code
    icd9_code_dict = None
    if cfg.load_pattern["icd_code9"]:
        print("Step-4: Searching ICD-9 code into text file..........")
        icd9_code_dict = highlighter.search_icd_code(txt_list, code_type="ICD-9")

    # Step-5: Get closet match of keyword impairment
    print("Step-5: Get closet match of keyword impairment..........")
    matched_keyword_dict = sent_extractor.get_matched_keyword_dict(txt_list)
    highlighter.set_impairment_keyword_dict(matched_keyword_dict)

    # Step-6: Get date time list from text file
    print("Step-6: Get date time list from text file..........")
    matched_date_dict = generic_matcher.get_match_date_dict(txt_list)
    matched_number_dict = generic_matcher.get_match_number_dict(txt_list)
    highlighter.set_page_matched_date_dict(matched_date_dict)
    highlighter.set_page_matched_number_dict(matched_number_dict)

    # Step-7: Highlighting ICD-10 code and all keyword impairment into pdf
    print("Step-7: Highlighting ICD-10 code and all keyword impairment into pdf............")
    pdf_output_file, json_code_output_file = highlighter.highlight_icd_code_and_keyword_impairment(icd10_code_dict, pdf_file_name=pdf_file_name)
    print(f"File[{pdf_output_file}] is saved after highlighting ICD-10 code and all keyword impairment ")
    print(f"Highlighted code and impairment coordinates are saved into [{json_code_output_file}] file.")

    # Step-8: Clean up: move the current file into processed folder
    if Path(f"{cfg.file_path['PROCESSED_PATH']}/{pdf_file}").exists():
        # take backup of existing file
        shutil.move(f"{cfg.file_path['PROCESSED_PATH']}/{pdf_file}",
                    f"{cfg.file_path['PROCESSED_PATH']}/{pdf_file}_bkp")
        # then move it
        shutil.move(pdf_file_name, cfg.file_path["PROCESSED_PATH"])
    else:
        shutil.move(pdf_file_name, cfg.file_path["PROCESSED_PATH"])

    # remove all pdf and text files
    purge(f"{cfg.file_path['TMP_PDF_FILES_PATH']}/*.pdf")
    purge(f"{cfg.file_path['TMP_TXT_FILES_PATH']}/*.txt")
    pdf_list = []
    txt_list = []

#####################Processing File[Practice_Copy_Test_OCR.pdf] for highlighting###########################
Step-1: Splitting pdf file............
Step-2: Extracting text from pdf............
Step-3: Searching ICD-10 code into text file..........
Step-5: Get closet match of keyword impairment..........
Step-6: Get date time list from text file..........
Step-7: Highlighting ICD-10 code and all keyword impairment into pdf............
coords: [(129.47999572753906, 616.4400024414062, 195.20997619628906, 628.4458618164062)]
coords: [(82.5602035522461, 254.08001708984375, 111.33020782470703, 262.08392333984375), (129.47999572753906, 575.1199951171875, 159.29995727539062, 585.1248779296875)]
coords: [(132.0, 171.79998779296875, 154.001220703125, 181.80487060546875), (168.0, 575.1199951171875, 189.3599853515625, 585.1248779296875)]
coords: [(168.0, 589.6400146484375, 184.48800659179688, 599.6448974609375)]
coords: [(282.9599914550781, 274.0799865722656, 339.53155517578125, 283.0843811035156)

In [17]:
!zip output.zip /home/ocreng/ocrhigh/output/*.*

  adding: home/ocreng/ocrhigh/output/Practice_Copy_Test_OCR_output.json (deflated 80%)
  adding: home/ocreng/ocrhigh/output/Practice_Copy_Test_OCR_output.pdf (deflated 3%)


In [18]:
#matched_date_dict

In [19]:
#matched_number_dict[6]

In [20]:
#matched_keyword_dict[0]

##Impairment Comparihension

In [ ]:
json_arr_list = list(map(sent_extractor.get_json_array_list, text_list))
tmp_wrong_keyword_dict = {
    idx: set([list(element.keys())[0] for element in json_arr if json_arr is not None])
    for idx, json_arr in enumerate(json_arr_list)
}

In [ ]:
json_arr_list

In [ ]:
doc_keyword_dict = {
    idx:  {{k : v for k, v in page_keyword_dict.items()} for page_keyword_dict in json_arr}
    for idx, json_arr in enumerate(json_arr_list) if json_arr is not None
}

In [ ]:
doc_keyword_dict[5]

[{'O2Sat': 'O2Sat'},
 {'BP:': 'BP:'},
 {'T:': 'T:'},
 {'BMI': 'BMI'},
 {'Ht:': 'Ht:'},
 {'tdap': 'tdap'},
 {'Ht': 'Ht'},
 {'%': '%'},
 {'O2Sat:': 'O2Sat:'},
 {'DOB': 'DOB'},
 {'NKDA': 'NKDA'},
 {'BP': 'BP'},
 {'covid': 'covid'},
 {'No medications reported by patient': 'no medications reported by patient'},
 {'diopathic osteoarthritis': 'idiopathic osteoarthritis'},
 {'Sleep pattern disturbance': 'sleep pattern disturbance'},
 {'OBSTETRICS & GYNECOLOGY': 'Obstetrics & Gynecology'},
 {'No medications reported': 'no medications reported'},
 {'Primary Care Provider': 'primary care provider'},
 {'Mixed hyperlipidemia': 'mixed hyperlipidemia'},
 {'Multinodular goiter': 'multinodular goiter'},
 {'Chronic neck pain': 'chronic neck pain'},
 {'Body mass index': 'body mass index'},
 {'Physical Exam': 'physical exam'},
 {'overweight': 'overweight'},
 {'Encounter': 'encounter'},
 {'Allergies': 'allergies'},
 {'Vaccines': 'vaccines'},
 {'shingles': 'shingles'},
 {'Headache': 'headache'},
 {'Patient'

In [ ]:
doc_keyword_dict = {}
for idx, json_arr in enumerate(json_arr_list):
  if json_arr is not None:
    page_keyword_dict = {}
    for element in json_arr:
      for k, v in element.items():
        page_keyword_dict[k] = v
    doc_keyword_dict[idx] = page_keyword_dict
doc_keyword_dict[0]

{'DOB': 'DOB',
 'Policy Number': 'Policy Number',
 'Patient': 'patient',
 'Name': 'Name'}

In [ ]:
doc_keyword_dict[5]

{'BMI': 'BMI',
 'O2Sat:': 'O2Sat:',
 'NKDA': 'NKDA',
 'tdap': 'tdap',
 'Ht:': 'Ht:',
 '%': '%',
 'BP:': 'BP:',
 'BP': 'BP',
 'O2Sat': 'O2Sat',
 'T:': 'T:',
 'Ht': 'Ht',
 'DOB': 'DOB',
 'covid': 'covid',
 'No medications reported by patient': 'no medications reported by patient',
 'diopathic osteoarthritis': 'idiopathic osteoarthritis',
 'Sleep pattern disturbance': 'sleep pattern disturbance',
 'OBSTETRICS & GYNECOLOGY': 'Obstetrics & Gynecology',
 'No medications reported': 'no medications reported',
 'Primary Care Provider': 'primary care provider',
 'Mixed hyperlipidemia': 'mixed hyperlipidemia',
 'Multinodular goiter': 'multinodular goiter',
 'Chronic neck pain': 'chronic neck pain',
 'Body mass index': 'body mass index',
 'Physical Exam': 'physical exam',
 'overweight': 'overweight',
 'Encounter': 'encounter',
 'Allergies': 'allergies',
 'Vaccines': 'vaccines',
 'shingles': 'shingles',
 'Headache': 'headache',
 'Patient': 'patient',
 'patient': 'patient',
 'Female': 'female',
 'Pu

In [ ]:
keyword_dict = doc_keyword_dict[5]
for keyword in keyword_dict.keys():
  print(keyword_dict[keyword])

BMI
O2Sat:
NKDA
tdap
Ht:
%
BP:
BP
O2Sat
T:
Ht
DOB
covid
no medications reported by patient
idiopathic osteoarthritis
sleep pattern disturbance
Obstetrics & Gynecology
no medications reported
primary care provider
mixed hyperlipidemia
multinodular goiter
chronic neck pain
body mass index
physical exam
overweight
encounter
allergies
vaccines
shingles
headache
patient
patient
female
pulse
Name
Appt
flu
Wt


In [ ]:
doc_keyword_dict["BMI"]

'BMI'

In [ ]:
json_arr_list[:3]

[[{'DOB': 'DOB'},
  {'Policy Number': 'Policy Number'},
  {'Patient': 'patient'},
  {'Name': 'Name'}],
 [],
 [{'US': 'US'},
  {'EKG': 'EKG'},
  {'progress notes': 'progress notes'},
  {'discharge': 'discharge'},
  {'Patient': 'patient'},
  {'patient': 'patient'}]]

In [ ]:
wrong_keyword_dict[5]

{'O2Sat': 'O2Sat',
 'BP:': 'BP:',
 'covid': 'covid',
 'Ht:': 'Ht:',
 'NKDA': 'NKDA',
 'tdap': 'tdap',
 'T:': 'T:',
 'BP': 'BP',
 'O2Sat:': 'O2Sat:',
 'Ht': 'Ht',
 'DOB': 'DOB',
 '%': '%',
 'BMI': 'BMI',
 'No medications reported by patient': 'no medications reported by patient',
 'diopathic osteoarthritis': 'idiopathic osteoarthritis',
 'Sleep pattern disturbance': 'sleep pattern disturbance',
 'OBSTETRICS & GYNECOLOGY': 'Obstetrics & Gynecology',
 'No medications reported': 'no medications reported',
 'Primary Care Provider': 'primary care provider',
 'Mixed hyperlipidemia': 'mixed hyperlipidemia',
 'Multinodular goiter': 'multinodular goiter',
 'Chronic neck pain': 'chronic neck pain',
 'Body mass index': 'body mass index',
 'hyperlipidemia': 'hyperlipidemia',
 'osteoarthritis': 'osteoarthritis',
 'Physical Exam': 'physical exam',
 'overweight': 'overweight',
 'Encounter': 'encounter',
 'Allergies': 'allergies',
 'neck pain': 'neck pain',
 'Vaccines': 'vaccines',
 'shingles': 'shingl

In [ ]:
#icd9_code_dict1[19]

##Split TXT/PDF

In [ ]:
!cp Practice_Copy_Test_OCR.pdf /home/ocreng/ocrhigh/input

In [ ]:
INPUT_PDF_FILES_PATH = "/home/ocreng/ocrhigh/input"
for pdf_file in os.listdir(INPUT_PDF_FILES_PATH):
  pdf_file_name = f"{INPUT_PDF_FILES_PATH}/{pdf_file}"

  # Step-1: splitting pdf file
  print("Step-1: Splitting pdf file............")
  pdf_list = split_pdf(highlighter.PDF_FILES_PATH, pdf_file_name)

  # Step-2: Extracting text from pdf
  print("Step-2: Extracting text from pdf............")
  txt_list = extract_text_from_pdf(highlighter.PDF_FILES_PATH, highlighter.TXT_FILES_PATH, pdf_list)

Step-1: Splitting pdf file............
Step-2: Extracting text from pdf............


In [ ]:
!zip text_files.zip /home/ocreng/ocrhigh/txt-files/*.*
!zip pdf_files.zip /home/ocreng/ocrhigh/pdf-files/*.*

In [ ]:
#!unzip pdf_files.zip

In [ ]:
#!cp /home/ocreng/ocrhigh/pdf-files/page-38.pdf .

##Split List

In [ ]:
def split_list(a_list, divider):
  half = len(a_list)//divider
  return a_list[:half], a_list[half:]

A = [1,2,3,4,5,6]
B, C = split_list(A, 3)
B, C

([1, 2], [3, 4, 5, 6])

In [ ]:
6 // 3

2

In [ ]:
int(4 / 2)

2

In [ ]:
!pip install more-itertools

In [ ]:
import more_itertools

In [ ]:
for batch in more_itertools.batched([1,2,3,4,5,6], 3):
  print(batch)

(1, 2, 3)
(4, 5, 6)


In [ ]:
for batch in more_itertools.batched("ABCDEFGHIJ", 4):
  print(batch)

In [ ]:
def split_list(a_list, divider):
  half = len(a_list) // divider
  return a_list[:half], a_list[half:]

In [ ]:
L =np.array([
(63.5, 166.5, 81.5, 175.50439453125),
(84.5, 166.5, 133.59002685546875, 175.50439453125),
(130.0, 264.0, 147.49998474121094, 273.00439453125),
(150.5, 264.0, 199.59002685546875, 273.00439453125),
(150.5, 264.0, 199.59002685546875, 273.00439453125),
(150.5, 264.0, 199.59002685546875, 273.00439453125)
])

In [ ]:
(63.5, 166.5, 133.59002685546875, 175.50439453125)
(130.0, 264.0, 199.59002685546875, 273.00439453125)

In [ ]:
np.split(L, 3)

[array([[ 63.5       , 166.5       ,  81.5       , 175.50439453],
        [ 84.5       , 166.5       , 133.59002686, 175.50439453]]),
 array([[130.        , 264.        , 147.49998474, 273.00439453],
        [150.5       , 264.        , 199.59002686, 273.00439453]]),
 array([[150.5       , 264.        , 199.59002686, 273.00439453],
        [150.5       , 264.        , 199.59002686, 273.00439453]])]

In [ ]:
LL= [
(23.5, 443.0, 77.00015258789062, 454.00537109375),
(79.5, 443.0, 122.99948120117188, 454.00537109375),
(126.5, 443.0, 142.1002197265625, 454.00537109375),
(384.5, 478.5, 430.0002136230469, 485.50341796875),
(195.49978637695312, 487.0, 240.9998321533203, 494.00341796875),
(246.5, 487.0, 260.5, 494.00341796875)]

In [ ]:
len(LL) // 3

2

In [ ]:
for batch_coord in more_itertools.batched(LL, 2):
  f_coord = batch_coord[0]
  l_coord = batch_coord[-1]
  print(f"f_coord: {f_coord}")
  print(f"l_coord: {l_coord}")
  phrase_coord = (f_coord[0], f_coord[1], l_coord[2], l_coord[3])
  print(f"key_phrase: CTA DISSECTION, phrase_coord: {phrase_coord}")

f_coord: (23.5, 443.0, 77.00015258789062, 454.00537109375)
l_coord: (79.5, 443.0, 122.99948120117188, 454.00537109375)
key_phrase: CTA DISSECTION, phrase_coord: (23.5, 443.0, 122.99948120117188, 454.00537109375)
f_coord: (126.5, 443.0, 142.1002197265625, 454.00537109375)
l_coord: (384.5, 478.5, 430.0002136230469, 485.50341796875)
key_phrase: CTA DISSECTION, phrase_coord: (126.5, 443.0, 430.0002136230469, 485.50341796875)
f_coord: (195.49978637695312, 487.0, 240.9998321533203, 494.00341796875)
l_coord: (246.5, 487.0, 260.5, 494.00341796875)
key_phrase: CTA DISSECTION, phrase_coord: (195.49978637695312, 487.0, 260.5, 494.00341796875)


In [ ]:
key_phrase_sent = "plan of care"
coordinates = [
    (303.35198974609375, 578.380859375, 350.51043701171875, 589.455078125),
    (432.93896484375, 647.3748779296875, 480.2064514160156, 658.4490966796875)]

phrase_len = len(key_phrase_sent.split())
print((len(coordinates) // phrase_len) )
for batch_coord in more_itertools.batched(coordinates, 3):
  f_coord = batch_coord[0]
  l_coord = batch_coord[-1]
  print(f"f_coord: {f_coord}")
  print(f"l_coord: {l_coord}")
  phrase_coord = (f_coord[0], f_coord[1], l_coord[2], l_coord[3])
  print(f"key_phrase: {key_phrase_sent}, phrase_coord: {phrase_coord}")

0
f_coord: (303.35198974609375, 578.380859375, 350.51043701171875, 589.455078125)
l_coord: (432.93896484375, 647.3748779296875, 480.2064514160156, 658.4490966796875)
key_phrase: plan of care, phrase_coord: (303.35198974609375, 578.380859375, 480.2064514160156, 658.4490966796875)


In [ ]:
LL= np.array([
(23.5, 443.0, 77.00015258789062, 454.00537109375),
(79.5, 443.0, 122.99948120117188, 454.00537109375),
(126.5, 443.0, 142.1002197265625, 454.00537109375),
(384.5, 478.5, 430.0002136230469, 485.50341796875),
(195.49978637695312, 487.0, 240.9998321533203, 494.00341796875),
(246.5, 487.0, 260.5, 494.00341796875)])

for batch_coord in np.split(LL, 3):
  f_coord = batch_coord[0]
  l_coord = batch_coord[-1]
  print(f"f_coord: {f_coord}")
  print(f"l_coord: {l_coord}")
  phrase_coord = (f_coord[0], f_coord[1], l_coord[2], l_coord[3])
  print(f"key_phrase: CTA DISSECTION, phrase_coord: {phrase_coord}")

f_coord: [ 23.5        443.          77.00015259 454.00537109]
l_coord: [ 79.5        443.         122.9994812  454.00537109]
key_phrase: CTA DISSECTION, phrase_coord: (23.5, 443.0, 122.99948120117188, 454.00537109375)
f_coord: [126.5        443.         142.10021973 454.00537109]
l_coord: [384.5        478.5        430.00021362 485.50341797]
key_phrase: CTA DISSECTION, phrase_coord: (126.5, 443.0, 430.0002136230469, 485.50341796875)
f_coord: [195.49978638 487.         240.99983215 494.00341797]
l_coord: [246.5        487.         260.5        494.00341797]
key_phrase: CTA DISSECTION, phrase_coord: (195.49978637695312, 487.0, 260.5, 494.00341796875)


```log
f_coord: (63.5, 166.5, 81.5, 175.50439453125)
l_coord: (84.5, 166.5, 133.59002685546875, 175.50439453125)
key_phrase: CTA DISSECTION, phrase_coord: (63.5, 166.5, 133.59002685546875, 175.50439453125)
f_coord: (130.0, 264.0, 147.49998474121094, 273.00439453125)
l_coord: (150.5, 264.0, 199.59002685546875, 273.00439453125)
key_phrase: CTA DISSECTION, phrase_coord: (130.0, 264.0, 199.59002685546875, 273.00439453125)
```

##PDF OCRing

In [ ]:
import pikepdf

def remove_password_from_pdf(input_file, output_file, password=None):
    pdf = pikepdf.open(input_file, password=password)
    pdf.save(output_file)

In [ ]:
remove_password_from_pdf("Keyword_Sample_File#4_Redacted_Redacted.pdf",
                         "Keyword_Sample_File#4_Redacted_Redacted_pw_removed.pdf", "synodex")

In [ ]:
!ocrmypdf --skip-text Sample_File_No3_Redacted_pw_removed.pdf Sample_File_No3_Redacted.pdf

Scanning contents: 100% 50/50 [00:00<00:00, 326.07page/s]
Start processing 2 pages concurrently
    1 skipping all processing on this page
    2 [tesseract] lots of diacritics - possibly poor OCR
   12 [tesseract] lots of diacritics - possibly poor OCR
   20 [tesseract] lots of diacritics - possibly poor OCR
   19 [tesseract] lots of diacritics - possibly poor OCR
   21 [tesseract] lots of diacritics - possibly poor OCR
   36 [tesseract] lots of diacritics - possibly poor OCR
   46 [tesseract] lots of diacritics - possibly poor OCR
OCR: 100% 50.0/50.0 [01:42<00:00,  2.05s/page]
Postprocessing...
PDF/A conversion: 100% 50/50 [00:17<00:00,  2.80page/s]
Recompressing JPEGs: 0image [00:00, ?image/s]
Deflating JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]
Optimize ratio: 1.00 savings: -0.2%
Image optimization did not improve the file - optimizations will not be used
Output file is a PDF/A-2B (as expected)


In [ ]:
!rm -rf Keyword_Sample_File4_Redacted_Redacted_pw_removed.pdf

##Exact Match

In [ ]:
mytext = """
  Encounter #16
History & Physical Report
1/25/2021: Lab Orders - Hypertriglyceridemia,                 sporadic (E78.3) (Marina Dobricic)
   ppointment:    1/25/2021   9:00 AM
                                  / Race: White
Male
 The patient is a 41 year old male.
 Assessment & Plan (Angela Sabbagh; 1/23/2021            11:36 AM)
 Hypertriglyceridemia, sporadic (E78.3)
 Impression: stopped will restart and check labs
Current Plans
       @   Comprehensive Metabolic Panel (CMP) (80053)
       @   Complete Blood Count with Differential (CBC) (85025)
       @   Lipid Panel (LP) (80061)
Signed by Marina Dobricic (1/27/2021        9:54 AM)
 Comprehensive Metabolic Panel (CMP) (80053) Final, Reviewed (Collected: 01/25/2021)
 Diagnosis: Hypertriglyceridemia, sporadic (E78.3)
 Note: Testing done at Silver Pine Medical         Group unless otherwise specified. 43455 Schoenherr Road, Suite 19, Sterling Heights, MI 48313
      Sodium                                  141 mmol/L              (Normal Range: 135-145 mmol/L)         Result Note:
                                                                    Resutt Annotation:
      Potassium                               4.1  mmol/L             (Normal Range: 3.5-5.2 mmol/L)        Result    Note:
                                                                    Result Annotation:
      Chloride                                103   mmol/L            (Normal Range: 98-107 mmol/L)         Result    Note:
                                                                    Result Annotation:
      Carbon    Dioxide (CO2)                 30 mmol/L               (Normal Range: 21-31    mmol/L)     Result     Note:
                                                                    Resutt Annotation:
      Anion Gap                               8                       (Normal Range: 5-17)      Result     Note:
                                                                    Resutt Annotation:
      Glucose                                 89 mg/dL                (Normal Range: 60-99 mg/dL)        Result     Note:
                                                                    Resutt Annotation:
      Blood Urea Nitrogen (BUN)               17 mg/dLl               (Normal Range: 8-23 mg/dL)        Result    Note:
                                                                    Resutt Annotation:
      Creatinine                              1.03   mg/dL            (Normal Range: 0.80-1.40 mg/dL)        Result     Note:
                                                                    Resutt Annotation:
      GFR                                     79 mL/min/1.73m2        (Normal Range: >59 mL/min/1.73m2)          Reeult      Note:
                                                                    Resutt Annotation:
      GFR   African American                  96 mL/min/1.73m2        (Normal Range: >59 mL/min/1.73m2)          Result      Note:
                                                                    Result Annotation:
      Calcium                                 9.0 mg/dL               (Normal Range: 8.5-11.0 mg/dL)        Result     Note;:
                                                                    Result Annotation:
      Protein Total                           7.8 g/dL                (Normal Range: 6.4-8.2 g/dL)      Result     Note:
                                                                    Resutt Annotation:
      Albumin                                 4.4 g/dL                (Normal Range: 3.4-5.0 g/dL)      Result     Note:
                                                                    Resutt Annotation:
      Globulin                                3.40                    (Normal Range: 2.20-4.00)       Result    Wote:
                                                                    Resutt Annotation:
      Album   in/Globulin Ratio               1.3                         Result Note:
                                                                    Resutt Annotation:
      Alkaline Phosphatase (ALP)              64 U/L                  (Normal Range: 50-116 U/L)       Result    Note:
                                                                    Resutt Annotation:
      Aspartate Aminotransferase (AST)        25  U/L                 (Normal Range: 15-37 U/L)      Result    Note:
                                                                    Resutt Annotation:
      Alanine Aminotransferase (ALT)          33 U/L                  (Normal Range: 16-63 U/L)      Reeult    Note:
                                                                    Resutt Annotation:
      Bilirubin Total                         0.5 mg/dL               (Normal Range:   0.3-1.2 mg/dL)     Result     Note:
                                                                    Resutt Annotation:
 Complete Blood Count with Differential (CBC) (85025) Final, Reviewed (Collected: 01/25/2021)
 Diagnosis: Hypertriglyceridemia, sporadic (E78.3)
 Note: Testina done at Silver Pine Medical Group unless otherwise specified. 43455 Schoenherr Road. Suite 19. Sterlina Heiahts. Ml 48313
06/28/2022 12:27 pm                                              s                                                                          Page 11/144
"""

In [ ]:
term = "MI"
#matches = len(re.findall(r'(?i)\bAlbumin\b', mytext)) > 0
# matches = len(re.findall(f'(?i)\\b{term}\\b', mytext)) > 0
#if len(re.findall(f'(?i)\\b{term}\\b', mytext)) > 0:
matches = re.findall(r'\bMI\b', "mi mister miss")
matches

[]

In [ ]:
with open("page-30.txt", "r") as f:
  mytext = f.read()
if len(re.findall(f'(?i)\\b{term}\\b', mytext)) > 0:
  print("Found")

FileNotFoundError: ignored

In [ ]:
wrong_keyword_dict1[0]

In [ ]:
icd10_code_dict1

In [ ]:
%%time

text_list = split_pdf(highlighter.PDF_FILES_PATH, "APS_38600000R_final.pdf")
len(text_list)

##Highlight Test

In [ ]:
pdf_file_name = "page-30.pdf"
pdf_file = fitz.open(pdf_file_name)
for page_num, page in enumerate(pdf_file):
  for keyword_impairment in wrong_keyword_dict1[0]:
    # print(f"keyword_impairment11: {keyword_impairment}")
    highlight = page.search_for(keyword_impairment)
    print(f"keyword_impairment: {keyword_impairment}, coords: {highlight}")
    highlight = page.add_highlight_annot(highlight)
    highlight.update()
output_pdf_file_name = f"{pdf_file_name.split('.')[0]}_output.pdf"
pdf_file.save(output_pdf_file_name, garbage=4, deflate=True, clean=True)

keyword_impairment: Urea, coords: [Rect(70.31328582763672, 435.0400390625, 83.75199890136719, 443.0439453125)]
keyword_impairment: Comprehensive Metabolic Panel, coords: [Rect(60.47419357299805, 207.52001953125, 118.06912994384766, 215.52392578125), Rect(121.66831970214844, 207.52001953125, 156.94491577148438, 215.52392578125), Rect(159.82464599609375, 207.52001953125, 176.2230682373047, 215.52392578125), Rect(29.997119903564453, 282.8800048828125, 95.99125671386719, 290.8839111328125), Rect(99.83040618896484, 282.8800048828125, 141.58639526367188, 290.8839111328125), Rect(144.46612548828125, 282.8800048828125, 163.6642608642578, 290.8839111328125)]
keyword_impairment: ALT, coords: [Rect(148.30589294433594, 665.4400634765625, 158.38491821289062, 673.4439697265625)]
keyword_impairment: GFR, coords: [Rect(45.355674743652344, 477.0400390625, 57.77448272705078, 485.0439453125), Rect(45.35564422607422, 498.1600341796875, 53.8148307800293, 506.1639404296875)]
keyword_impairment: CBC, coords:

##ICD 10 Code

In [ ]:
!mkdir -p pdf-files
!mkdir -p txt-files

In [ ]:
# define directory path after creating it
pdf_files_path = "pdf-files"
txt_files_path = "txt-files"

# create nlp instance
nlp = English()


def split_pdf(pdf_path):
  pdf_in_file = open(pdf_path, "rb")
  pdf = PdfReader(pdf_in_file)
  pdf_list = []
  for page in range(len(pdf.pages)):
      inputpdf = PdfReader(pdf_in_file)
      output = PdfWriter()
      output.add_page(inputpdf.pages[page])
      with open(f"{pdf_files_path}/page-{page}.pdf", "wb") as outputStream:
          output.write(outputStream)
          pdf_list.append(f"page-{page}.pdf")
  return pdf_list


def extract_text_from_pdf(pdf_list):
    txt_file_list = []
    i = 0
    for pdf_file in pdf_list:
        with open(os.path.join(pdf_files_path, pdf_file), "rb") as f:
            pdf = pdftotext.PDF(f)

        # Read all the text into one string
        pdf_text = "\n\n".join(pdf)

        # write text into file
        with open(f"{txt_files_path}/page-{str(i)}.txt", "a") as f:
            f.write(pdf_text)
        txt_file_list.append(f"{txt_files_path}/page-{str(i)}.txt")
        i += 1
    return txt_file_list


def get_opt_pattern(icd_10_code):
  # create alternate pattern
  code_arr = icd_10_code.split(".")
  if len(code_arr) > 1:
    code1 = f"{code_arr[0]}. {code_arr[1]}"
    code2 = f"{code_arr[0]} .{code_arr[1]}"
    code3 = f"{code_arr[0]} . {code_arr[1]}"
    return [code1, code2, code3]
  else:
    return icd_10_code


def highlight_icd10_code(pdf_page_dict: dict, pdf_file_name: str):
    pdf_file = fitz.open(pdf_file_name)

    def highlight_pdf(highlight):
        for inst in highlight:
          highlight = page.add_highlight_annot(inst)
          highlight.update()
          highlight = page.search_for(text_to_be_highlighted)
          print(f"Page-{page_num}: ", code, highlight, end='\n')

    for page_num, page in enumerate(pdf_file):
        if page_num in pdf_page_dict:
          for code in pdf_page_dict[page_num]:
            text_to_be_highlighted = code
            highlight = page.search_for(text_to_be_highlighted)
            print(f"Page-{page_num}: ", code, highlight, end='\n')
            if len(highlight) == 0:
                alternate_code_list = get_opt_pattern(code)
                for alt_code in alternate_code_list:
                  text_to_be_highlighted = alt_code
                  highlight = page.search_for(text_to_be_highlighted)
                  # highlight pdf for option pattern
                  highlight_pdf(highlight)
            # highlight pdf for main pattern
            highlight_pdf(highlight)

    output_pdf_file_name = f"{pdf_file_name.split('.')[0]}_output.pdf"
    pdf_file.save(output_pdf_file_name, garbage=4, deflate=True, clean=True)
    return output_pdf_file_name


def search_icd_10_code(txt_list):
  pdf_page_vocab = {}
  for txt_file in txt_list:
    with open(txt_file, "r") as f:
      page_txt = f.read()
      # filter the page that have line number instead of code
      if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
        doc = nlp(page_txt)
        code_list = [ent.text for ent in doc.ents]
        if len(code_list) != 0:
          #print(txt_file)
          page_number = int(txt_file.split("/")[1].split(".")[0].split("-")[1])
          pdf_page_vocab[page_number] = code_list
          # print(f"Page[{txt_file.split('/')[1]}]: {code_list}")
  return pdf_page_vocab

In [ ]:
# Step-1: splitting pdf file
pdf_file_name = "28page.pdf"
pdf_list = split_pdf(pdf_file_name)

# Step-2: Extracting text from pdf
txt_list = extract_text_from_pdf(pdf_list)

# Step-3: loading and updating patterns to Spacy
nlp.add_pipe("entity_ruler").from_disk("./icd10_code_patterns-v1.jsonl")

# Step-4: Searching ICD-10 code
#print (txt_list)
pdf_page_vocab = search_icd_10_code(txt_list)

# Step-5: Highlighting ICD-10 code into pdf
output_file_name = highlight_icd10_code(pdf_page_vocab, pdf_file_name)
print(f"File[{output_file_name}] is saved after highlighting ICD-10 code")

##Phrase matching

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
synid_df = pd.read_csv(cfg.csv_files["IMP_CSV"])
synid_df.head()

,SynId,Short_Description
0,KW20262,US thyroid
1,KW20261,no graphic evidence of malignancy
2,KW20260,no evidence of malignancy
3,KW20259,scattered areas of fibroglandular density
4,KW20258,scattered areas of fibroglandular density


In [ ]:
def make_icd_10_keyword_pattern(synid_df):
  patterns = []
  for _, row in synid_df.iterrows():
    patterns.append(row["Short_Description"])
  return patterns

In [ ]:
keywords = make_icd_10_keyword_pattern(synid_df)
keywords[:10]

['US thyroid',
 'no graphic evidence of malignancy',
 'no evidence of malignancy',
 'scattered areas of fibroglandular density',
 'scattered areas of fibroglandular density',
 'BI-RADS 6',
 'BI-RADS 5',
 'BI-RADS 4C',
 'BI-RADS 4B',
 'BI-RADS 4A']

In [ ]:
nlp = spacy.load('en_core_web_sm')
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.tokenizer.pipe(keywords))
phrase_matcher.add('keywords', patterns)

In [ ]:
with open("page-2.txt", "r") as f:
  page_txt = f.read()
  # filter the page that have line number instead of code
  #if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
  doc = nlp(page_txt)
  matches = phrase_matcher(doc)

  keyword_list = []
  for match_id, start, end in matches:
    span = doc[start: end]
    keyword_list.append(f"{span}")

In [ ]:
keyword_list

['T', 'HHS', 'US']

In [ ]:
with open("page-25.txt", "r") as f:
  page_txt = f.read()
  # filter the page that have line number instead of code
  #if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
  doc = nlp(page_txt)
  matches = phrase_matcher(doc)

  keyword_list = []
  for match_id, start, end in matches:
    span = doc[start: end]
    keyword_list.append(f"{span}")

In [ ]:
keyword_list

['Hypertriglyceridemia',
 'Hypertriglyceridemia',
 'EKG',
 'Hypertension',
 'Atrial Fibrillation',
 'Headache']

In [ ]:
with open("page-36.txt", "r") as f:
  page_txt = f.read()
  # filter the page that have line number instead of code
  #if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
  doc = nlp(page_txt)
  matches = phrase_matcher(doc)

  keyword_list = []
  for match_id, start, end in matches:
    span = doc[start: end]
    keyword_list.append(f"{span}")

In [ ]:
keyword_list

['Hypertriglyceridemia',
 'Hypertriglyceridemia',
 'EKG',
 'Hypertension',
 'Atrial Fibrillation',
 'Diplopia',
 'Headache',
 'Visual Loss',
 'Gynecomastia',
 'Dysphagia',
 'Hematuria']

In [ ]:
synid_df = pd.read_csv("synid_and_keywords_impairment.csv")
keywords = make_icd_10_keyword_pattern(synid_df)

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.tokenizer.pipe(keywords))
phrase_matcher.add('keywords', patterns)

In [ ]:
with open("page-36.txt", "r") as f:
  page_txt = f.read()
  # filter the page that have line number instead of code
  #if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
  doc = nlp(page_txt)
  matches = phrase_matcher(doc)

  keyword_list = []
  for match_id, start, end in matches:
    span = doc[start: end]
    keyword_list.append(f"{span}")

In [ ]:
keyword_list

['Hypertriglyceridemia',
 'Hypertriglyceridemia',
 'EKG',
 'Hypertension',
 'Atrial Fibrillation',
 'Diplopia',
 'Headache',
 'Visual Loss',
 'Gynecomastia',
 'Dysphagia',
 'Hematuria',
 'Headaches']

In [ ]:
wrong_keyword_dict1[30]

{'ALP',
 'ALT',
 'AST',
 'Albumin',
 'Alkaline Phosphatase',
 'Anion Gap',
 'Aspartate',
 'BUN',
 'Bilirubin Total',
 'Blood Urea Nitrogen',
 'CBC',
 'CMP',
 'CO2',
 'Calcium',
 'Chloride',
 'Complete Blood Count',
 'Comprehensive Metabolic Panel',
 'Creatinine',
 'Encounter',
 'GFR',
 'Glucose',
 'Hypertriglyceridemia',
 'Lipid Panel',
 'MI',
 'Potassium',
 'Sodium',
 'Urea',
 'Urea Nitrogen',
 'ppointment'}

In [ ]:
keywords = [keyword for keyword in wrong_keyword_dict1[30]]
len(keywords)

29

In [ ]:
len(list(set(keywords)))

29

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.tokenizer.pipe(keywords))
phrase_matcher.add('keywords', patterns)

In [ ]:
with open("page-30.txt", "r") as f:
  page_txt = f.read()
  # filter the page that have line number instead of code
  #if not re.search("(P[ ][0-9]+)(,\s)(L[0-9]+)", page_txt):
  doc = nlp(page_txt)
  matches = phrase_matcher(doc)

  keyword_list = []
  for match_id, start, end in matches:
    span = doc[start: end]
    keyword_list.append(f"{span}")

print(len(keyword_list))
len(list(set(keyword_list)))

37


29

In [ ]:

phrase_matcher = PhraseMatcher(nlp.vocab)

keywords = [keyword for keyword in wrong_keyword_dict1[30]]
pattern1 = [{'ORTH': keywords}]
phrase_matcher.add('keywords', None, pattern1)

TypeError: ignored